In [ ]:
#importing the libraries required for the evaluation
import numpy as np
import rasterio
from pygeotools.lib import malib, warplib, iolib,geolib, timelib
import numpy.ma as ma
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
from imview.lib import pltlib
from osgeo import gdal, ogr

### Reading RED and SWIR Bands of Landsat ETM+

In [ ]:
fn1='LE71480371999229AGS00_B3.TIF'
fn2='LE71480371999229AGS00_B5.TIF'
fn_list=[fn1,fn2]
ds_list = warplib.memwarp_multi_fn(fn_list, extent='intersection', res='max', t_srs=fn1)

In [ ]:
band_ratio=ma.asarray(red,dtype=np.float32)/ma.asarray(swir,dtype=np.float32)

In [ ]:
malib.print_stats(band_ratio)

### Limiting Band Ratio of Red/SWIR to 2.2 (Kaab et al. 2012) 

In [ ]:
debris=ma.masked_outside(band_ratio,0, 2.2)

### Eliminating scattered Pixels as Noise using the mask_island function (malib, pygeotools)
## Be careful on the number of itereations, higher iterations might filter out valid continous data near edges as well

In [ ]:
Debris_Islands_masked=malib.mask_islands(debris,iterations=2)

In [ ]:
out_fn = 'Debris_cleaned_2.tif' 
iolib.writeGTiff(Debris_Islands_masked, out_fn, ds_list[1]) #Debris Cover Map with isolated pixels removed

### Reading shapefiles as updatable Features, Entering the percentage Debris covered per feature into a newly created field "ECW"

In [ ]:
source = ogr.Open('Drung_drung.shp', update=True)
layer = source.GetLayer()
layer_defn = layer.GetLayerDefn()
new_field = ogr.FieldDefn("ECW", ogr.OFTReal)
layer.CreateField(new_field)
source = None

In [ ]:
glac_shp_fn='Drung_drung.shp'
glac_shp_ds = ogr.Open(glac_shp_fn, update=True) #shapefile_dataset
glac_shp_lyr = glac_shp_ds.GetLayer() #layer of shapefile, all the operations are performed on it....
glac_shp_srs = glac_shp_lyr.GetSpatialRef() #Spatial_reference_of_shapefile
feat_count = glac_shp_lyr.GetFeatureCount() #Feature_Count
print("Input glacier polygon count: %i" % feat_count)


In [ ]:
print str(glac_shp_srs)

In [ ]:
#dh_dt='WV_Carto_RateMaps-tile-0-last.tif' #input elevation_difference_map
#dh_dt='Filtered_dh_1_5_IQR.tif' #input elevation_difference_map
#z='B-tile-0.tif' #input DEM
#dh_dt_ds=gdal.Open(dh_dt) 
#z_ds=gdal.Open(z)
Landsat_geom = geolib.ds_geom_intersection(ds_list, t_srs=glac_shp_srs) #intersecting_extent
glac_shp_lyr.SetSpatialFilter(Landsat_geom) #spatial filtering of shapefiles within extent
feat_count = glac_shp_lyr.GetFeatureCount() #now only the polygons within the extents are considered
print("Filtered glacier polygon count: %i" % feat_count)
glac_shp_lyr.ResetReading()

In [ ]:
debris_fieldname = "Percentage_Final"
glacnum_fieldname = "RGIId"

In [ ]:
for n, feat in enumerate(glac_shp_lyr):
    print'Hi' #Check to see program flow...
    #Extracting glacier id/name from input shapefiles
    glacnum = feat.GetField(glacnum_fieldname)
    glacnum = float(glacnum.split('-')[-1])
    glacnum_fmt = '%0.5f'
    feat_fn = str(glacnum)
    print("\n%i of %i: %s\n" % (n+1, feat_count, feat_fn))
    glac_geom = feat.GetGeometryRef()
    glac_geom.AssignSpatialReference(glac_shp_srs)
    glac_geom_extent = geolib.geom_extent(glac_geom)
    glac_area = glac_geom.GetArea()
    print str(glac_area)
    #warping the Landsat_Band and Debris_islanded into common extents
    ds_list1 = warplib.memwarp_multi_fn([fn1, out_fn], res='max',r='bilinear',
            extent=glac_geom_extent, t_srs='EPSG:32643', verbose=False)
    glac_geom_mask = geolib.geom2mask(glac_geom, ds_list1[0])
    total_pixel = ma.array(iolib.ds_getma(ds_list1[0]), mask=glac_geom_mask)
    debris_pixel = ma.array(iolib.ds_getma(ds_list1[1]), mask=glac_geom_mask) 
    ds_res = geolib.get_res(ds_list[0])
    valid_Total_area = total_pixel.count()*ds_res[0]*ds_res[1]
    valid_Debris_area = debris_pixel.count()*ds_res[0]*ds_res[1]
    Debris_area_perc = (valid_Debris_area/valid_Total_area)*100
    print("Total_Area ="+str(valid_Total_area))
    print("Total_Debris_Area="+str(valid_Debris_area))
    print("Debris_Area_Percentage="+str(Debris_area_perc))
    glac_shp_lyr.SetFeature(feat)
    feat.SetField("ECW", Debris_area_perc)
    glac_shp_lyr.SetFeature(feat)
    #feat.SetField(debris_fieldname, Debris_area_perc)
    #layer.SetFeature(feat)
glac_shp_ds=None